In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('../datasets/mushrooms.csv')
df.head(15)

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


In [4]:
np.unique(df["cap_shape"].values)

array(['b', 'c', 'f', 'k', 's', 'x'], dtype=object)

In [19]:
le = LabelEncoder()
df = df.apply(le.fit_transform)

In [20]:
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [21]:
df.shape

(8124, 23)

In [22]:
data = df.values
X_data = data[:,1:]
y_data = data[:,0]

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3)

In [24]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5686, 22) (5686,)
(2438, 22) (2438,)


In [25]:
np.unique(y_train)

array([0, 1])

In [26]:
class NaiveBayesClassifier:
    
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        
    def prior_probability(self, klass):
        prob = np.sum(self.y_train == klass) / self.y_train.shape[0]
        return prob
    
    def conditional_probability(self, klass, feature_name, feature_value):
        X_desired = self.X_train[self.y_train == klass]
        numerator = np.sum(X_desired[:, feature_name] == feature_value)
        denominator = np.sum(self.y_train == klass)
        prob = numerator / denominator
        return prob
    
    def classify(self, x):
        posterior = []
        for klass in np.unique(self.y_train):
            likelihood = 1
            for f in range(self.X_train.shape[1]):
                likelihood *= self.conditional_probability(klass, f, x[f])
            prior = self.prior_probability(klass)
            
            posterior.append(likelihood * prior)
        return posterior
    
    def predict(self, x):
        probabilities = self.classify(x)
        return np.array(probabilities).argmax()
    
    def get_predictions(self, X_test):
        predictions = list()
        for x in X_test:
            predictions.append(self.predict(x))
        return predictions
    
    def accuracy(self, preds, y_test):
        return np.mean(preds==y_test)

In [27]:
nb = NaiveBayesClassifier(X_train, y_train)

In [28]:
preds = nb.get_predictions(X_test)

In [29]:
nb.accuracy(preds, y_test)

0.9979491386382281

In [30]:
print(nb.predict(X_test[1]))

print(y_test[1])

1
1
